# Flat Image Net - Basic Graph


In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import shutil
from utils.data import init_dir

## Load data

In [12]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

Extracting /data/fashion/train-images-idx3-ubyte.gz
Extracting /data/fashion/train-labels-idx1-ubyte.gz
Extracting /data/fashion/t10k-images-idx3-ubyte.gz
Extracting /data/fashion/t10k-labels-idx1-ubyte.gz


## Basic summary functions

In [13]:
def variable_summaries(name, var):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))

        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


def img_summaries(name, var):
    with tf.name_scope(name):
        tf.summary.image(name, var)

## Custom layer functions

In [14]:
def simple_layer(name, input_data, shape, activation='linear'):
    w_name = 'w_' + name
    b_name = 'b_' + name
    if activation == 'relu':
        w = tf.get_variable(w_name, shape=shape, initializer=tf.contrib.layers.variance_scaling_initializer())
    else:
        w = tf.get_variable(w_name, shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0.1), shape=shape[1])
    
    variable_summaries(w_name+'summary', w)
    variable_summaries(b_name+'summary', bias)
    
    output_data = tf.matmul(input_data, w) + bias
    if activation == 'relu':
        output_data = tf.nn.relu(output_data)
    elif activation == 'sigmoid':
        output_data = tf.nn.sigmoid(output_data)
    elif activation == 'tanh':
        output_data = tf.nn.tanh(output_data)
    return output_data

## Build Net Graph

In [15]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('flat_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')

    
    with tf.variable_scope('simple_layer_1'):
        raw_prediction = simple_layer(name='layer1', input_data=images, shape=[784, 64], activation='relu')
        
    with tf.variable_scope('simple_layer_2'):
        raw_prediction = simple_layer(name='layer2', input_data=raw_prediction, shape=[64, 10])
        raw_prediction = tf.nn.dropout(raw_prediction, keep_dropout_prob)
            
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)
    
    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', cross_entropy_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)       
        
    with tf.name_scope('training'):
        train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
            
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [19]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/flat_image_net/'

logging_meta = init_model_logging(base_dir, 'experiment_final', graph=graph, remove_existing=True)
logging_meta

{'model_path': '/tensorboard_summaries/flat_image_net/experiment_final/valid/model.ckpt',
 'saver': <tensorflow.python.training.saver.Saver at 0x7f647a244e48>,
 'train_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f647d5c4c88>,
 'train_writer_dir': '/tensorboard_summaries/flat_image_net/experiment_final/train',
 'valid_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f647a244e10>,
 'valid_writer_dir': '/tensorboard_summaries/flat_image_net/experiment_final/valid'}

## Run Net

In [20]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']
        
with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    for iteration in range(5000):
        ##################
        # Training phase #
        ##################
        _images, _labels = data.train.next_batch(100)
        _ = session.run([train_step], feed_dict={images: _images, labels: _labels, keep_dropout_prob: 0.5})
        if iteration % 10 == 0:
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss],
                                                     feed_dict={images: _images, 
                                                                labels: _labels, 
                                                                keep_dropout_prob: 1.0})
            logging_meta['train_writer'].add_summary(_summary, iteration)
            print("Train Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
      
        ####################
        # Validation phase #
        ####################
        if iteration % 100 == 0:
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss], 
                                          feed_dict={images: data.validation.images, 
                                                     labels: data.validation.labels,
                                                     keep_dropout_prob: 1.0})
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))
            
    _prediction, = session.run([prediction], feed_dict={images: data.validation.images, keep_dropout_prob: 1.0})

Train Iteration 0: loss 2.2709972858428955, accuracy 0.25
= Valid Iteration 0: loss 2.284444570541382, accuracy 0.19280000030994415 =
Train Iteration 10: loss 2.155974864959717, accuracy 0.4699999988079071
Train Iteration 20: loss 2.0102171897888184, accuracy 0.5799999833106995
Train Iteration 30: loss 1.9878098964691162, accuracy 0.6200000047683716
Train Iteration 40: loss 1.9136526584625244, accuracy 0.6499999761581421
Train Iteration 50: loss 1.928338646888733, accuracy 0.6800000071525574
Train Iteration 60: loss 1.8527419567108154, accuracy 0.7599999904632568
Train Iteration 70: loss 1.8700753450393677, accuracy 0.6700000166893005
Train Iteration 80: loss 1.868169903755188, accuracy 0.6899999976158142
Train Iteration 90: loss 1.7584320306777954, accuracy 0.8100000023841858
Train Iteration 100: loss 1.7553436756134033, accuracy 0.8100000023841858
= Valid Iteration 100: loss 1.7972677946090698, accuracy 0.7483999729156494 =
Train Iteration 110: loss 1.8584342002868652, accuracy 0.699

Train Iteration 1010: loss 1.625801682472229, accuracy 0.8700000047683716
Train Iteration 1020: loss 1.6776559352874756, accuracy 0.7900000214576721
Train Iteration 1030: loss 1.6533770561218262, accuracy 0.8199999928474426
Train Iteration 1040: loss 1.6938087940216064, accuracy 0.7900000214576721
Train Iteration 1050: loss 1.6208652257919312, accuracy 0.8999999761581421
Train Iteration 1060: loss 1.6786025762557983, accuracy 0.8100000023841858
Train Iteration 1070: loss 1.6056900024414062, accuracy 0.8799999952316284
Train Iteration 1080: loss 1.63197660446167, accuracy 0.8500000238418579
Train Iteration 1090: loss 1.6645147800445557, accuracy 0.800000011920929
Train Iteration 1100: loss 1.6651898622512817, accuracy 0.7900000214576721
= Valid Iteration 1100: loss 1.6421338319778442, accuracy 0.8345999717712402 =
Train Iteration 1110: loss 1.6392730474472046, accuracy 0.8600000143051147
Train Iteration 1120: loss 1.6655279397964478, accuracy 0.8100000023841858
Train Iteration 1130: los

Train Iteration 2050: loss 1.6668922901153564, accuracy 0.7799999713897705
Train Iteration 2060: loss 1.6077446937561035, accuracy 0.8600000143051147
Train Iteration 2070: loss 1.5894012451171875, accuracy 0.8799999952316284
Train Iteration 2080: loss 1.5864545106887817, accuracy 0.8999999761581421
Train Iteration 2090: loss 1.6169078350067139, accuracy 0.8600000143051147
Train Iteration 2100: loss 1.6190426349639893, accuracy 0.8500000238418579
= Valid Iteration 2100: loss 1.6202490329742432, accuracy 0.8515999913215637 =
Train Iteration 2110: loss 1.551416039466858, accuracy 0.9399999976158142
Train Iteration 2120: loss 1.6384567022323608, accuracy 0.8399999737739563
Train Iteration 2130: loss 1.6799495220184326, accuracy 0.7900000214576721
Train Iteration 2140: loss 1.5872163772583008, accuracy 0.8799999952316284
Train Iteration 2150: loss 1.6238675117492676, accuracy 0.8399999737739563
Train Iteration 2160: loss 1.6375668048858643, accuracy 0.8299999833106995
Train Iteration 2170: 

= Valid Iteration 3100: loss 1.607574462890625, accuracy 0.8664000034332275 =
Train Iteration 3110: loss 1.6125413179397583, accuracy 0.8500000238418579
Train Iteration 3120: loss 1.6512138843536377, accuracy 0.800000011920929
Train Iteration 3130: loss 1.6150741577148438, accuracy 0.8299999833106995
Train Iteration 3140: loss 1.6136364936828613, accuracy 0.8700000047683716
Train Iteration 3150: loss 1.5905641317367554, accuracy 0.8899999856948853
Train Iteration 3160: loss 1.6124299764633179, accuracy 0.8500000238418579
Train Iteration 3170: loss 1.5980294942855835, accuracy 0.8899999856948853
Train Iteration 3180: loss 1.5968362092971802, accuracy 0.8799999952316284
Train Iteration 3190: loss 1.6006295680999756, accuracy 0.8799999952316284
Train Iteration 3200: loss 1.628790020942688, accuracy 0.8500000238418579
= Valid Iteration 3200: loss 1.6059350967407227, accuracy 0.8611999750137329 =
Train Iteration 3210: loss 1.6302132606506348, accuracy 0.8600000143051147
Train Iteration 3220

Train Iteration 4140: loss 1.623616337776184, accuracy 0.8500000238418579
Train Iteration 4150: loss 1.6162246465682983, accuracy 0.8700000047683716
Train Iteration 4160: loss 1.6064748764038086, accuracy 0.8500000238418579
Train Iteration 4170: loss 1.6207950115203857, accuracy 0.8399999737739563
Train Iteration 4180: loss 1.619407057762146, accuracy 0.8600000143051147
Train Iteration 4190: loss 1.6020264625549316, accuracy 0.8600000143051147
Train Iteration 4200: loss 1.5719330310821533, accuracy 0.8999999761581421
= Valid Iteration 4200: loss 1.6000298261642456, accuracy 0.8646000027656555 =
Train Iteration 4210: loss 1.6273232698440552, accuracy 0.8500000238418579
Train Iteration 4220: loss 1.57769775390625, accuracy 0.8899999856948853
Train Iteration 4230: loss 1.6209845542907715, accuracy 0.8399999737739563
Train Iteration 4240: loss 1.608399510383606, accuracy 0.8500000238418579
Train Iteration 4250: loss 1.6461328268051147, accuracy 0.8299999833106995
Train Iteration 4260: loss

In [18]:
_prediction

array([[  1.07458769e-16,   2.02757429e-12,   1.86062967e-14, ...,
          1.20290244e-09,   6.35634545e-10,   1.00000000e+00],
       [  1.00000000e+00,   1.08944589e-10,   4.81477025e-10, ...,
          3.60207895e-19,   1.18872046e-09,   1.29780964e-10],
       [  4.83380526e-01,   6.40578568e-02,   2.49041594e-03, ...,
          1.40850883e-04,   2.82091321e-04,   3.37777566e-03],
       ..., 
       [  4.12653719e-07,   1.89303489e-06,   5.06804872e-06, ...,
          3.06115195e-04,   1.09589295e-04,   8.92834098e-04],
       [  1.57973474e-08,   4.50984352e-08,   1.33971754e-03, ...,
          5.15389363e-11,   9.98659015e-01,   3.85884574e-10],
       [  1.13414526e-05,   7.33688765e-04,   1.22960819e-05, ...,
          5.74003971e-05,   4.08170272e-05,   2.81546563e-05]], dtype=float32)